# <font color = 'dodgerblue'>**PreProcessing_Feature_Emgineering_IMDB**
In this notebook, we will do basic_proprpcessing followed by Featuyre Engineering.

In preprocessing, we will do following:

(1) Remove HTML Tags
(2) Remove emails
(3) Remove URLs

After Preprocessing, we will add following faetures:

  1. number of words
  2. number of characters
  3. number of characters without space
  4. average word length
  5. number of numbers (tokens that are numbers)
  6. number of sentences
  6. number of nouns or propernouns
  7. number of aux
  8. number of verbs
  9. number of adjectives
  10. number of ner (entiites)

Finally, we will craete classes which we can reuse for (1) general preprocessing and (2) manual faeture extraction. We will make sure our classes are compatible with sklearn pipelines.

# <font color = 'dodgerblue'>**Install/Import Libraries**

In [1]:
# install spacy
if 'google.colab' in str(get_ipython()):
    !pip install -U spacy -qq



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 980.7 kB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [33]:
# Import the Path module from the pathlib library
from pathlib import Path

# Import the pandas library for working with data frames
import pandas as pd
import numpy as np

# Import the spacy library for natural language processing
import spacy

# Import the List type from the typing module to use in function annotations
from typing import List

from bs4 import BeautifulSoup
import re

from pprint import pprint

In [3]:
# check spacy version
spacy.__version__


'3.7.2'

# <font color = 'dodgerblue'>**Data**

In [56]:
text = ["""New version of operation system is iOS 11. It is better than iOS 9.
The new version of iPhone X seems cool.""", """ <p> The video of iphone x released. I liked iOS 9 but I like iOS 11 more.
You may not like my like @Tech_Guru #Iphone #IOS harpreet@utdallas.edu  https://jindal.utdallas.edu/""",
        """</p><p>The concept of regular expressions began in the 1950s, when the American mathematician <a href="/wiki/Stephen_Cole_Kleene" title="Stephen Cole Kleene">Stephen Cole Kleene</a> formalized the description of a <i><a href="/wiki/Regular_language" title="Regular language">regular language</a></i>. They came into common use with <a href="/wiki/Unix" title="Unix">Unix</a> text-processing utilities. Different <a href="/wiki/Syntax_(programming_languages)" title="Syntax (programming languages)">syntaxes</a> for writing regular expressions have existed since the 1980s, one being the <a href="/wiki/POSIX" title="POSIX">POSIX</a> standard and another, widely used, being the <a href="/wiki/Perl" title="Perl">Perl</a> syntax.
</p><p>Regular expressions are used in <a href="/wiki/Search_engine" title="Search engine">search engines</a>, search and replace dialogs of <a href="/wiki/Word_processor" title="Word processor">word processors</a> and <a href="/wiki/Text_editor" title="Text editor">text editors</a>, in <a href="/wiki/Text_processing" title="Text processing">text processing</a> utilities such as <a href="/wiki/Sed" title="Sed">sed</a> and <a href="/wiki/AWK" title="AWK">AWK</a> and in <a href="/wiki/Lexical_analysis" title="Lexical analysis">lexical analysis</a>. Many <a href="/wiki/Programming_language" title="Programming language">programming languages</a> provide regex capabilities either built-in or via <a href="/wiki/Library_(computing)" title="Library (computing)">libraries</a>, as it has uses in many situations.
</p> """]
df = pd.DataFrame(text, columns=['Reviews'])

In [57]:
df.head()

,Reviews
0,New version of operation system is iOS 11. It ...
1,<p> The video of iphone x released. I liked i...
2,</p><p>The concept of regular expressions bega...


In [59]:
pprint(df['Reviews'].values[0], width = 80)

('New version of operation system is iOS 11. It is better than iOS 9.\n'
 'The new version of iPhone X seems cool.')


In [60]:
pprint(df['Reviews'].values[1], width = 80)

(' <p> The video of iphone x released. I liked iOS 9 but I like iOS 11 more.\n'
 'You may not like my like @Tech_Guru #Iphone #IOS harpreet@utdallas.edu  '
 'https://jindal.utdallas.edu/')


In [160]:
pprint(df['Reviews'].values[2], width = 80)

('</p><p>The concept of regular expressions began in the 1950s, when the '
 'American mathematician <a href="/wiki/Stephen_Cole_Kleene" title="Stephen '
 'Cole Kleene">Stephen Cole Kleene</a> formalized the description of a <i><a '
 'href="/wiki/Regular_language" title="Regular language">regular '
 'language</a></i>. They came into common use with <a href="/wiki/Unix" '
 'title="Unix">Unix</a> text-processing utilities. Different <a '
 'href="/wiki/Syntax_(programming_languages)" title="Syntax (programming '
 'languages)">syntaxes</a> for writing regular expressions have existed since '
 'the 1980s, one being the <a href="/wiki/POSIX" title="POSIX">POSIX</a> '
 'standard and another, widely used, being the <a href="/wiki/Perl" '
 'title="Perl">Perl</a> syntax.\n'
 '</p><p>Regular expressions are used in <a href="/wiki/Search_engine" '
 'title="Search engine">search engines</a>, search and replace dialogs of <a '
 'href="/wiki/Word_processor" title="Word processor">word processors</a> a

In [65]:
type(df['Reviews'].values[2])

str

# <font color = 'dodgerblue'>**Import Spacy Model**

In [9]:
# check the models we have dowloaded in spacy folder
!python -m spacy download en_core_web_sm

2024-01-22 05:19:36.086570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 05:19:36.086621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 05:19:36.087885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 05:19:37.232673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 28.4 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.6.0
    Uninstalling en-core-web-sm-3.6.0:
      Successfully uninstall

In [173]:
# We will load the model -en_core_web_sm
nlp = spacy.load('en_core_web_sm')


# <font color = 'dodgerblue'>**Pre-processing**

In [174]:
def basic_clean(text: str) -> str:
    """
    This function performs basic text cleaning on an input string by removing HTML tags (if present)
    and replacing newline and return characters with a space.

    Parameters:
    text (str): The input string to be cleaned

    Returns:
    str: The cleaned string
    """
    # Use BeautifulSoup to remove HTML tags (if present)
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Replace newline and return characters with a space
    return re.sub(r'[\n\r]',' ', text)

In [175]:
cleaned_text= [basic_clean(text).encode('utf-8', 'ignore').decode() for text in df['Reviews'].values]

In [176]:
pprint(cleaned_text[2])

('The concept of regular expressions began in the 1950s, when the American '
 'mathematician Stephen Cole Kleene formalized the description of a regular '
 'language. They came into common use with Unix text-processing utilities. '
 'Different syntaxes for writing regular expressions have existed since the '
 '1980s, one being the POSIX standard and another, widely used, being the Perl '
 'syntax. Regular expressions are used in search engines, search and replace '
 'dialogs of word processors and text editors, in text processing utilities '
 'such as sed and AWK and in lexical analysis. Many programming languages '
 'provide regex capabilities either built-in or via libraries, as it has uses '
 'in many situations.  ')


In [177]:
# initialize an empty list to store processed text
processed_text = []

# initialize an empty list to store processed text
processed_text = []

# Use context manager to temporarily disable the named pipes of spaCy NLP processing pipeline
with nlp.select_pipes(disable=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']):
  # process multiple documents in parallel using the spaCy NLP library
  for doc in nlp.pipe(cleaned_text, batch_size=3, n_process=1):
      # filter out URLs and emails, then extract text of each token
      tokens = [token.text for token in doc if not token.like_url and not token.like_email]
      text = ' '.join(tokens)
      # join tokens back into a single string and add to the processed_text list
      processed_text.append(text)

In [178]:
cleaned_text[1]

'  The video of iphone x released. I liked iOS 9 but I like iOS 11 more. You may not like my like @Tech_Guru #Iphone #IOS harpreet@utdallas.edu  https://jindal.utdallas.edu/'

In [179]:
processed_text[1]

'   The video of iphone x released . I liked iOS 9 but I like iOS 11 more . You may not like my like @Tech_Guru # Iphone # IOS  '

In [183]:
nlp = spacy.load('en_core_web_sm')
prefixes = list(nlp.Defaults.prefixes)
prefixes += ['@']
prefixes.remove(r'#')

# Compile prefix regex based on selected prefixes
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [184]:
# initialize an empty list to store processed text
processed_text = []

# Use context manager to temporarily disable the named pipes of spaCy NLP processing pipeline
with nlp.select_pipes(disable=['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']):
  # process multiple documents in parallel using the spaCy NLP library
  for doc in nlp.pipe(cleaned_text, batch_size=3, n_process=1):
      # filter out URLs and emails, then extract text of each token
      tokens = [token.text for token in doc if not token.like_url and not token.like_email]
      text = ' '.join(tokens)
      # join tokens back into a single string and add to the processed_text list
      processed_text.append(text)

In [182]:
processed_text[1]

'   The video of iphone x released . I liked iOS 9 but I like iOS 11 more . You may not like my like @ Tech_Guru #Iphone #IOS  '

In [204]:
from sklearn.base import BaseEstimator, TransformerMixin
from bs4 import BeautifulSoup
import re
import spacy
import numpy as np
from nltk.stem.porter import PorterStemmer
import os

class SpacyPreprocessor(BaseEstimator, TransformerMixin):

    def __init__(self, model, *, batch_size = 64, lemmatize=True, lower=True, remove_stop=True,
                remove_punct=True, remove_email=True, remove_url=True, remove_num=False, stemming = False,
                add_user_mention_prefix=True, remove_hashtag_prefix=False, basic_clean_only=False):

        self.model = model
        self.batch_size = batch_size
        self.remove_stop = remove_stop
        self.remove_punct = remove_punct
        self.remove_num = remove_num
        self.remove_url = remove_url
        self.remove_email = remove_email
        self.lower = lower
        self.add_user_mention_prefix = add_user_mention_prefix
        self.remove_hashtag_prefix = remove_hashtag_prefix
        self.basic_clean_only = basic_clean_only

        if lemmatize and stemming:
            raise ValueError("Only one of 'lemmatize' and 'stemming' can be True.")

        self.lemmatize = lemmatize
        self.stemming = stemming

    def basic_clean(self, text):
        soup = BeautifulSoup(text, "html.parser")
        text = soup.get_text()
        text = re.sub(r'[\n\r]', ' ', text)
        return text.strip()

    def spacy_preprocessor(self, texts):
        final_result = []
        nlp = spacy.load(self.model)

        # Disable unnecessary pipelines in spaCy model
        if self.lemmatize:
            # Disable parser and named entity recognition
            disabled_pipes = ['parser', 'ner']
        else:
            # Disable tagger, parser, attribute ruler, lemmatizer and named entity recognition
            disabled_pipes = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

        with nlp.select_pipes(disable=disabled_pipes):
          # Modify tokenizer behavior based on user_mention_prefix and hashtag_prefix settings
          if self.add_user_mention_prefix or self.remove_hashtag_prefix:
              prefixes = list(nlp.Defaults.prefixes)
              if self.add_user_mention_prefix:
                  prefixes += ['@']  # Treat '@' as a separate token
              if self.remove_hashtag_prefix:
                  prefixes.remove(r'#')  # Don't separate '#' from the following text
              prefix_regex = spacy.util.compile_prefix_regex(prefixes)
              nlp.tokenizer.prefix_search = prefix_regex.search

          # Process text data in parallel using spaCy's nlp.pipe()
          for doc in nlp.pipe(texts, batch_size=self.batch_size):
              filtered_tokens = []
              for token in doc:
                  # Check if token should be removed based on specified filters
                  if self.remove_stop and token.is_stop:
                      continue
                  if self.remove_punct and token.is_punct:
                      continue
                  if self.remove_num and token.like_num:
                      continue
                  if self.remove_url and token.like_url:
                      continue
                  if self.remove_email and token.like_email:
                      continue

                  # Append the token's text, lemma, or stemmed form to the filtered_tokens list
                  if self.lemmatize:
                      filtered_tokens.append(token.lemma_)
                  elif self.stemming:
                      filtered_tokens.append(PorterStemmer().stem(token.text))
                  else:
                      filtered_tokens.append(token.text)

              # Join the tokens and apply lowercasing if specified
              text = ' '.join(filtered_tokens)
              if self.lower:
                  text = text.lower()
              final_result.append(text.strip())

        return final_result


    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        try:
            if not isinstance(X, (list, np.ndarray)):
                raise TypeError(f'Expected list or numpy array, got {type(X)}')

            x_clean = [self.basic_clean(text).encode('utf-8', 'ignore').decode() for text in X]

            # Check if only basic cleaning is required
            if self.basic_clean_only:
                return x_clean  # Return the list of basic-cleaned texts

            x_clean_final = self.spacy_preprocessor(x_clean)
            return x_clean_final

        except Exception as error:
            print(f'An exception occurred: {repr(error)}')


In [205]:
# import spacy pre-processor from custom module
preprocessor = SpacyPreprocessor(model='en_core_web_sm', batch_size=64, lemmatize=False, lower=False,
                                    remove_stop=False, remove_punct=False, remove_email=False,
                                    remove_url=False, remove_num=False, stemming=False,
                                    add_user_mention_prefix=False, remove_hashtag_prefix=False, basic_clean_only=True)

In [225]:
cleaned_text = preprocessor.fit_transform(df['Reviews'].values)

for item in cleaned_text:
    print()
    pprint(item, width = 80)


('New version of operation system is iOS 11. It is better than iOS 9. The new '
 'version of iPhone X seems cool.')

('The video of iphone x released. I liked iOS 9 but I like iOS 11 more. You '
 'may not like my like @Tech_Guru #Iphone #IOS harpreet@utdallas.edu  '
 'https://jindal.utdallas.edu/')

('The concept of regular expressions began in the 1950s, when the American '
 'mathematician Stephen Cole Kleene formalized the description of a regular '
 'language. They came into common use with Unix text-processing utilities. '
 'Different syntaxes for writing regular expressions have existed since the '
 '1980s, one being the POSIX standard and another, widely used, being the Perl '
 'syntax. Regular expressions are used in search engines, search and replace '
 'dialogs of word processors and text editors, in text processing utilities '
 'such as sed and AWK and in lexical analysis. Many programming languages '
 'provide regex capabilities either built-in or via libraries, as it has u

# Feature Extraction

In [207]:
noun_count = [] # create a list to store the noun count for each document
aux_count = [] # create a list to store the auxiliary verb count for each document
verb_count = [] # create a list to store the verb count for each document
adj_count =[] # create a list to store the adjective count for each document

# disable lemmatizer and named entity recognizer
disabled_pipes =  ['lemmatizer', 'ner']

# iterate over the documents in the dataframe using the spacy pipe method
with nlp.select_pipes(disable=disabled_pipes):
  for doc in nlp.pipe(cleaned_text, batch_size=3, n_process=1):

      # find all nouns and proper nouns in the document and store in a list
      nouns = [token.text for token in doc if (token.pos_ in ["NOUN","PROPN"])]

      # find all auxiliary verbs in the document and store in a list
      auxs =  [token.text for token in doc if (token.pos_ in ["AUX"])]

      # find all verbs in the document and store in a list
      verbs =  [token.text for token in doc if (token.pos_ in ["VERB"])]
      print(verbs)

      # find all adjectives in the document and store in a list
      adjectives =  [token.text for token in doc if (token.pos_ in ["ADJ"])]

      # store the count of nouns in the noun_count list
      noun_count.append(len(nouns))

      # store the count of auxiliary verbs in the aux_count list
      aux_count.append(len(auxs))

      # store the count of verbs in the verb_count list
      verb_count.append(len(verbs))

      # store the count of adjectives in the adj_count list
      adj_count.append(len(adjectives))


['seems']
['released', 'liked', 'like', 'like']
['began', 'formalized', 'came', 'processing', 'writing', 'existed', 'used', 'replace', 'sed', 'provide', 'built']


In [208]:
noun_count, verb_count, aux_count, adj_count

([6, 8, 40], [1, 4, 11], [2, 1, 5], [4, 1, 12])

In [211]:
nlp = spacy.load('en_core_web_sm')
disabled_pipes = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
with nlp.select_pipes(disable=disabled_pipes):
  if not nlp.has_pipe('sentencizer'):
    nlp.add_pipe('sentencizer')

    list_count_words = []
    list_count_characters = []
    list_count_characters_no_space = []
    list_avg_word_length = []
    list_count_digits = []
    list_count_numbers = []
    list_count_sentences = []

    for doc in nlp.pipe(cleaned_text, batch_size=3, n_process=1):
        # Count words (tokens)
        count_word = len([token for token in doc if not token.is_punct])

        # Count all characters (including spaces)
        count_char = len(doc.text)

        # Count characters without spaces
        count_char_no_space = len(doc.text_with_ws.replace(' ', ''))

        # Calculate average word length
        avg_word_length = count_char_no_space / (count_word + 1)

        # Count numbers (consecutive digits)
        count_numbers = len([token for token in doc if token.is_digit])

        # Count sentences
        count_sentences = len(list(doc.sents))

        list_count_words.append(count_word)
        list_count_characters.append(count_char)
        list_count_characters_no_space.append(count_char_no_space)
        list_avg_word_length.append(avg_word_length)
        list_count_numbers.append(count_numbers)
        list_count_sentences.append(count_sentences)

In [212]:
list_count_words,  list_count_characters, list_count_characters_no_space, list_avg_word_length, list_count_numbers, list_count_sentences

([22, 28, 106],
 [107, 170, 687],
 [86, 143, 584],
 [3.739130434782609, 4.931034482758621, 5.457943925233645],
 [2, 2, 0],
 [3, 3, 5])

In [214]:
for item in cleaned_text:
    print()
    pprint(item, width = 80)


('New version of operation system is iOS 11. It is better than iOS 9. The new '
 'version of iPhone X seems cool.')

('The video of iphone x released. I liked iOS 9 but I like iOS 11 more. You '
 'may not like my like @Tech_Guru #Iphone #IOS harpreet@utdallas.edu  '
 'https://jindal.utdallas.edu/')

('The concept of regular expressions began in the 1950s, when the American '
 'mathematician Stephen Cole Kleene formalized the description of a regular '
 'language. They came into common use with Unix text-processing utilities. '
 'Different syntaxes for writing regular expressions have existed since the '
 '1980s, one being the POSIX standard and another, widely used, being the Perl '
 'syntax. Regular expressions are used in search engines, search and replace '
 'dialogs of word processors and text editors, in text processing utilities '
 'such as sed and AWK and in lexical analysis. Many programming languages '
 'provide regex capabilities either built-in or via libraries, as it has u

In [229]:
nlp = spacy.load('en_core_web_sm')
count_ner = []
disabled_pipes = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']
with nlp.select_pipes(disable=disabled_pipes):
# Disable the tok2vec, tagger, parser, attribute ruler, and lemmatizer pipelines for improved performance
  for doc in nlp.pipe(cleaned_text, batch_size=1000, n_process=-1):
      ner_text = [ent.text for ent in doc.ents]
      print(ner_text)
      ner_labels = [ent.label_ for ent in doc.ents]
      print(ner_labels)
      count_ner.append(len(ner_labels))

['11', '9']
['CARDINAL', 'CARDINAL']
['9', '11', 'IOS']
['CARDINAL', 'CARDINAL', 'ORG']
['the 1950s', 'American', 'Stephen Cole', 'the 1980s', 'one', 'POSIX', 'Perl', 'AWK']
['DATE', 'NORP', 'PERSON', 'DATE', 'CARDINAL', 'ORG', 'ORG', 'ORG']


In [230]:
count_ner

[2, 3, 8]

In [235]:
from sklearn.base import TransformerMixin, BaseEstimator
import numpy as np
import spacy
import re
import sys
import os
from pathlib import Path


class ManualFeatures(TransformerMixin, BaseEstimator):

    """A transformer class for extracting manual features from text data.

    This class is designed to be used in a scikit-learn pipeline. It uses the spaCy
    library to extract a variety of manual features from text data, such as
    part-of-speech (POS) features, named entity recognition (NER) features,
    and count-based features.
    """



    def __init__(self, spacy_model, batch_size = 64, pos_features = True, ner_features = True, count_features = True):

        """
        Initialize the feature extractor.

        Parameters
        ----------
        spacy_model : str
            The name of the spaCy model to use for feature extraction.
        pos_features : bool, optional (default=True)
            Whether to extract part-of-speech (POS) features from the text data.
        ner_features : bool, optional (default=True)
            Whether to extract named entity recognition (NER) features from the text data.
        count_features : bool, optional (default=True)
            Whether to extract count-based features from the text data.
        """

        self.spacy_model = spacy_model
        self.batch_size = batch_size
        self.pos_features = pos_features
        self.ner_features = ner_features
        self.count_features = count_features

    def get_cores(self):
        """
        Get the number of CPU cores to use in parallel processing.
        """
        # Get the number of CPU cores available on the system.
        num_cores = os.cpu_count()
        if num_cores < 3:
            use_cores = 1
        else:
            use_cores = num_cores // 2 + 1
        return num_cores

    def get_pos_features(self, cleaned_text):

        nlp = spacy.load(self.spacy_model)
        noun_count = []
        aux_count = []
        verb_count = []
        adj_count =[]

        # Disable the lemmatizer and NER pipelines for improved performance
        disabled_pipes = ['lemmatizer', 'ner']
        with nlp.select_pipes(disable=disabled_pipes):
            n_process = self.get_cores()
            for doc in nlp.pipe(cleaned_text, batch_size=self.batch_size, n_process=n_process):
                # Extract nouns, auxiliaries, verbs, and adjectives from the document
                nouns = [token.text for token in doc if token.pos_ in ["NOUN","PROPN"]]
                auxs =  [token.text for token in doc if token.pos_ in ["AUX"]]
                verbs =  [token.text for token in doc if token.pos_ in ["VERB"]]
                adjectives =  [token.text for token in doc if token.pos_ in ["ADJ"]]

                # Store the count of each type of word in separate lists
                noun_count.append(len(nouns))
                aux_count.append(len(auxs))
                verb_count.append(len(verbs))
                adj_count.append(len(adjectives))

        # Stack the count lists vertically to form a 2D numpy array
        return np.transpose(np.vstack((noun_count, aux_count, verb_count, adj_count)))



    def get_ner_features(self, cleaned_text):
        nlp = spacy.load(self.spacy_model)
        count_ner = []

        # Disable the tok2vec, tagger, parser, attribute ruler, and lemmatizer pipelines for improved performance
        disabled_pipes = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer']
        with nlp.select_pipes(disable=disabled_pipes):
            n_process = self.get_cores()
            for doc in nlp.pipe(cleaned_text, batch_size=self.batch_size, n_process=n_process):
                ners = [ent.label_ for ent in doc.ents]
                count_ner.append(len(ners))

        # Convert the list of NER counts to a 2D numpy array
        return np.array(count_ner).reshape(-1, 1)


    def get_count_features(self, cleaned_text):
        list_count_words = []
        list_count_characters = []
        list_count_characters_no_space = []
        list_avg_word_length = []
        list_count_digits = []
        list_count_numbers = []
        list_count_sentences = []

        nlp = spacy.load(self.spacy_model)
        disabled_pipes = ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
        with nlp.select_pipes(disable=disabled_pipes):
            if not nlp.has_pipe('sentencizer'):
                nlp.add_pipe('sentencizer')
            n_process = self.get_cores()
            for doc in nlp.pipe(cleaned_text, batch_size=self.batch_size, n_process=n_process):
                count_word = len([token for token in doc if not token.is_punct])
                count_char = len(doc.text)
                count_char_no_space = len(doc.text_with_ws.replace(' ', ''))
                avg_word_length = count_char_no_space / (count_word + 1)
                count_numbers = len([token for token in doc if token.is_digit])
                count_sentences = len(list(doc.sents))

                list_count_words.append(count_word)
                list_count_characters.append(count_char)
                list_count_characters_no_space.append(count_char_no_space)
                list_avg_word_length.append(avg_word_length)
                list_count_numbers.append(count_numbers)
                list_count_sentences.append(count_sentences)

        count_features = np.vstack((list_count_words, list_count_characters, list_count_characters_no_space, list_avg_word_length,
                                    list_count_numbers, list_count_sentences))
        return np.transpose(count_features)


    def fit(self, X, y=None):
        """
        Fit the feature extractor to the input data.

        This method does not actually do any fitting, as the feature extractor is stateless.
        It simply returns the instance of the class.

        Parameters:
        X (list or numpy.ndarray): The input data.
        y (list or numpy.ndarray, optional): The target labels. Not used in this implementation.

        Returns:
        FeatureExtractor: The instance of the class.
        """
        return self


    def transform(self, X, y=None):
        """
        Transform the input data into a set of features.

        Parameters:
        X (list or numpy.ndarray): The input data.
        y (list or numpy.ndarray, optional): The target labels. Not used in this implementation.

        Returns:
        tuple: A tuple containing a 2D numpy array with shape (len(X), num_features) where num_features is the number of features extracted and a list of feature names.

        Raises:
        TypeError: If the input data is not a list or numpy array.
        Exception: If an error occurs while transforming the data into features.
        """
        try:
            # Check if the input data is a list or numpy array
            if not isinstance(X, (list, np.ndarray)):
                raise TypeError(f"Expected list or numpy array, got {type(X)}")


            feature_names = []

            if self.count_features:
                count_features = self.get_count_features(X)
                feature_names.extend(['count_words', 'count_characters',
                                      'count_characters_no_space', 'avg_word_length',
                                      'count_numbers', 'count_sentences'])
            else:
                count_features = np.empty(shape=(0, 0))

            if self.pos_features:
                pos_features = self.get_pos_features(X)
                feature_names.extend(['noun_count', 'aux_count', 'verb_count', 'adj_count'])
            else:
                pos_features = np.empty(shape=(0, 0))

            if self.ner_features:
                ner_features = self.get_ner_features(X)
                feature_names.extend(['ner'])
            else:
                ner_features = np.empty(shape=(0, 0))

            # Stack the feature arrays horizontally to form a single 2D numpy array
            return np.hstack((count_features, pos_features,  ner_features)), feature_names

        except Exception as error:
            print(f'An exception occured: {repr(error)}')

In [236]:
featurizer = ManualFeatures(spacy_model='en_core_web_sm', batch_size =3)

In [237]:
X_train_features, feature_names = featurizer.fit_transform(cleaned_text )

In [238]:
pd.DataFrame(X_train_features, columns=feature_names )

,count_words,count_characters,count_characters_no_space,avg_word_length,count_numbers,count_sentences,noun_count,aux_count,verb_count,adj_count,ner
0,22.0,107.0,86.0,3.739130,2.0,3.0,6.0,2.0,1.0,4.0,2.0
1,28.0,170.0,143.0,4.931034,2.0,3.0,8.0,1.0,4.0,1.0,3.0
2,106.0,687.0,584.0,5.457944,0.0,5.0,40.0,5.0,11.0,12.0,8.0
